# Installation Notes for K8s (minikube) and Kubeflow PIPELINES Locally on WSL Ubuntu

# Kubeflow Pipelines

## Docker
https://docs.docker.com/engine/install/ubuntu/
```
$ sudo apt-get remove docker docker-engine docker.io containerd runc

$ sudo apt-get update

$ sudo apt-get install apt-transport-https ca-certificates curl gnupg lsb-release

$ curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg

$ echo "deb [arch=amd64 signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null

$ sudo apt-get update
$ sudo apt-get install docker-ce docker-ce-cli containerd.io

$ sudo dockerd
$ docker run hello-world
```

## Kubernetes cluster (minikube)
https://kubernetes.io/docs/tutorials/hello-minikube/
```
# https://minikube.sigs.k8s.io/docs/start/

curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64
sudo install minikube-linux-amd64 /usr/local/bin/minikube

sudo dockerd

minikube start --cpus 12 --memory 16g --disk-size 40g
watch -c minikube kubectl -- get po -A

#alias kubectl='minikube kubectl'

minikube dashboard
```

## Kubeflow Pipelines
https://www.kubeflow.org/docs/components/pipelines/overview/pipelines-overview/
```
export PIPELINE_VERSION=0.5.0
# https://minikube.sigs.k8s.io/docs/handbook/kubectl/
minikube kubectl -- apply -k "github.com/kubeflow/pipelines/manifests/kustomize/cluster-scoped-resources?ref=$PIPELINE_VERSION"
minikube kubectl -- wait --for condition=established --timeout=60s crd/applications.app.k8s.io
minikube kubectl -- apply -k "github.com/kubeflow/pipelines/manifests/kustomize/env/dev?ref=$PIPELINE_VERSION"

watch -c minikube kubectl -- get po -A

minikube kubectl -- port-forward -n kubeflow svc/ml-pipeline-ui 8585:80
# Open: http://localhost:8585/

!!! Images are not available and minikube cannot pull them... check in some time in the docs (see link above)

#after ca. 20 minutes
#export NAMESPACE=istio-system
#kubectl port-forward -n ${NAMESPACE} svc/istio-ingressgateway 8585:80
# Open: http://localhost:8585/
```

## Cleaning
```
minikube pause
minikube unpause
minikube stop

minikube delete --all
```

In [ ]:
# env/platform-agnostic-pns hasn't been publically released, so you will install it from master
export PIPELINE_VERSION=1.7.0
minikube kubectl -- apply -k "github.com/kubeflow/pipelines/manifests/kustomize/cluster-scoped-resources?ref=$PIPELINE_VERSION"
minikube kubectl -- wait --for condition=established --timeout=60s crd/applications.app.k8s.io
minikube kubectl -- apply -k "github.com/kubeflow/pipelines/manifests/kustomize/env/platform-agnostic-pns?ref=$PIPELINE_VERSION"

In [ ]:
#minikube kubectl -- api-resources
minikube kubectl -- get pods
minikube kubectl -- describe

In [ ]:
# https://kubernetes.io/docs/tasks/configure-pod-container/pull-image-private-registry/
docker login
cat ~/.docker/config.json
sudo minikube kubectl -- create secret generic regcred \
    --from-file=.dockerconfigjson="~/.docker/config.json" \
    --type=kubernetes.io/dockerconfigjson

minikube kubectl -- create secret docker-registry regcred \
    --docker-server=<your-registry-server> \
    --docker-username=<your-name> \
    --docker-password=<your-pword> \
    --docker-email=<your-email>
        
minikube kubectl -- get secret regcred --output=yaml

wget -O my-private-reg-pod.yaml https://k8s.io/examples/pods/private-reg-pod.yaml
vi my-private-reg-pod.yaml

minikube kubectl -- apply -f my-private-reg-pod.yaml
minikube kubectl -- get pod private-reg

In [ ]:
export PIPELINE_VERSION=1.7.0
minikube kubectl -- apply -k "github.com/kubeflow/pipelines/manifests/kustomize/cluster-scoped-resources"
minikube kubectl -- wait --for condition=established --timeout=60s crd/applications.app.k8s.io
minikube kubectl -- apply -k "github.com/kubeflow/pipelines/manifests/kustomize/env/platform-agnostic"

kustomization.yaml
